In [1]:
import nltk
import pandas as pd
import numpy as np
from pymystem3 import Mystem
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from collections import Counter

In [2]:
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

#Preprocess function
def tokenization(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    text = " ".join(tokens)
    return text

def top_words(file_name):
    words = []
    f = open(file_name)
    txt = ''
    for line in f:
        txt = txt + line.replace('\n', '')
    
    array = tokenization(txt).split(' ')
    words = words + array

    return dict(Counter(words).most_common())

def make_dataframe(file_name):
    part = []
    data = top_words(file_name)
    df = pd.DataFrame.from_dict(data, orient='index', columns=['count'])
    df = df.drop(index=['', '–', '«','»'])
    
    for i in df.index:
        try:
            part = part + [nltk.pos_tag([i], lang='rus')[0][1]]
        except:
            pass
    
    df['part'] = part
    return df

def versus(chislitel):
    vs_putin = []
    vs_ryzan = []
    vs_tula = []
    vs_tumen = []
    vs_yamal = []
    
    for i in chislitel.index:
        ch = chislitel.loc[i][0]

        try:
            zn_pu = putin_df.loc[i][0]
        except:
            zn_pu = 100000
            
        try:
            zn_ry = ryzan_df.loc[i][0]
        except:
            zn_ry = 100000
        
        try:
            zn_tul = tula_df.loc[i][0]
        except:
            zn_tul = 100000
        
        try:
            zn_tum = tumen_df.loc[i][0]
        except:
            zn_tum = 100000
        
        try:
            zn_ya = yamal_df.loc[i][0]
        except:
            zn_ya = 100000

        vs_putin = vs_putin + [ch/zn_pu]
        vs_ryzan = vs_ryzan + [ch/zn_ry]
        vs_tula = vs_tula + [ch/zn_tul]
        vs_tumen = vs_tumen + [ch/zn_tum]
        vs_yamal = vs_yamal + [ch/zn_ya]
    
    while len(vs_putin)> len(chislitel):
        vs_putin = vs_putin[:-1]
    while len(vs_ryzan)> len(chislitel):
        vs_ryzan = vs_ryzan[:-1]
    while len(vs_tula)> len(chislitel):
        vs_tula = vs_tula[:-1]
    while len(vs_tumen)> len(chislitel):
        vs_tumen = vs_tumen[:-1]
    while len(vs_yamal)> len(chislitel):
        vs_yamal = vs_yamal[:-1]
                
    chislitel['vs_putin'] = vs_putin
    chislitel['vs_ryzan'] = vs_ryzan
    chislitel['vs_tula'] = vs_tula
    chislitel['vs_tumen'] = vs_tumen
    chislitel['vs_yamal'] = vs_yamal
    
def print_res(df,k):
    total = ''
    for i in df.index[:k]:
        total = total + i + ' - ' + str(df.loc[i]['count'])+ ', '
    print(total)

In [3]:
putin_df = make_dataframe('putin.txt')
versus(putin_df)
putin_df.head()

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
это,166,S-PRO,1.0,0.00166,0.00166,0.00166,0.00166
год,97,S,1.0,0.00097,0.00097,0.00097,0.00097
весь,95,A-PRO=m,1.0,0.00095,0.00095,0.00095,0.00095
наш,91,A-PRO=m,1.0,0.00091,0.00091,0.00091,0.00091
россия,85,S,1.0,0.00085,0.00085,0.00085,0.00085


In [4]:
#топ прилагательных
putin_df[(putin_df['part'] == 'A') | (putin_df['part'] == 'A=m')
        | (putin_df['part'] == 'A=f') | (putin_df['part'] == 'A=n') | (putin_df['part'] == 'A=sg')
        | (putin_df['part'] == 'A=pl') | (putin_df['part'] == 'A-PRO=pl') | (putin_df['part'] == 'A=comp2')].head(15)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
новый,69,A=m,1.0,0.00069,0.00069,0.00069,0.00069
должный,61,A=m,1.0,0.00061,0.00061,0.00061,0.00061
современный,35,A=m,1.0,0.00035,0.00035,0.00035,0.00035
важный,27,A=m,1.0,0.00027,0.00027,0.00027,0.00027
стратегический,26,A=m,1.0,0.00026,0.00026,0.00026,0.00026
ядерный,26,A=m,1.0,0.00026,0.00026,0.00026,0.00026
российский,21,A=m,1.0,0.00021,0.00021,0.00021,0.00021
большой,20,A=m,1.0,0.00020,0.00020,0.00020,0.00020
уважаемый,19,A=m,1.0,0.00019,0.00019,0.00019,0.00019
высокий,19,A=m,1.0,0.00019,0.00019,0.00019,0.00019


In [5]:
print_res(putin_df[(putin_df['part'] == 'A') | (putin_df['part'] == 'A=m')
        | (putin_df['part'] == 'A=f') | (putin_df['part'] == 'A=n') | (putin_df['part'] == 'A=sg')
        | (putin_df['part'] == 'A=pl') | (putin_df['part'] == 'A-PRO=pl') | (putin_df['part'] == 'A=comp2')].head(16), 16)

новый - 69, должный - 61, современный - 35, важный - 27, стратегический - 26, ядерный - 26, российский - 21, большой - 20, уважаемый - 19, высокий - 19, экономический - 17, военный - 15, социальный - 14, технологический - 14, должно - 13, прошлый - 12, 


In [6]:
#топ существительных
putin_df[(putin_df['part'] == 'S') | (putin_df['part'] == 'S-PRO')].head(16)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
это,166,S-PRO,1.0,0.00166,0.00166,0.00166,0.00166
год,97,S,1.0,0.00097,0.00097,0.00097,0.00097
россия,85,S,1.0,0.00085,0.00085,0.00085,0.00085
система,56,S,1.0,0.00056,0.00056,0.00056,0.00056
страна,55,S,1.0,0.00055,0.00055,0.00055,0.00055
человек,52,S,1.0,0.00052,0.00052,0.00052,0.00052
развитие,47,S,1.0,0.00047,0.00047,0.00047,0.00047
оружие,35,S,1.0,0.00035,0.00035,0.00035,0.00035
мир,29,S,1.0,0.00029,0.00029,0.00029,0.00029
возможность,29,S,1.0,0.00029,0.00029,0.00029,0.00029


In [7]:
print_res(putin_df[(putin_df['part'] == 'S') | (putin_df['part'] == 'S-PRO')].head(16), 16)

это - 166, год - 97, россия - 85, система - 56, страна - 55, человек - 52, развитие - 47, оружие - 35, мир - 29, возможность - 29, миллион - 28, уровень - 27, задача - 26, рост - 26, процент - 25, гражданин - 24, 


In [8]:
#топ глаголов
putin_df[(putin_df['part'] == 'V')].head(16)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
обеспечивать,31,V,1.0,0.00031,0.00031,0.00031,0.00031
сделать,31,V,1.0,0.00031,0.00031,0.00031,0.00031
сказать,27,V,1.0,0.00027,0.00027,0.00027,0.00027
создавать,26,V,1.0,0.00026,0.00026,0.00026,0.00026
позволять,23,V,1.0,0.00023,0.00023,0.00023,0.00023
становиться,22,V,1.0,0.00022,0.00022,0.00022,0.00022
хотеть,21,V,1.0,0.00021,0.00021,0.00021,0.00021
говорить,20,V,1.0,0.00020,0.00020,0.00020,0.00020
работать,19,V,1.0,0.00019,0.00019,0.00019,0.00019
идти,18,V,1.0,0.00018,0.00018,0.00018,0.00018


In [9]:
print_res(putin_df[(putin_df['part'] == 'V')].head(16), 16)

обеспечивать - 31, сделать - 31, сказать - 27, создавать - 26, позволять - 23, становиться - 22, хотеть - 21, говорить - 20, работать - 19, идти - 18, иметь - 17, решать - 14, знать - 14, сша - 14, получать - 13, выходить - 13, 


In [10]:
#топ наречий
putin_df[(putin_df['part'] == 'ADV')].head(8)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
сегодня,33,ADV,1.0,0.00033,0.00033,0.00033,0.00033
также,24,ADV,1.0,0.00024,0.00024,0.00024,0.00024
мочь,22,ADV,1.0,0.00022,0.00022,0.00022,0.00022
очень,20,ADV,1.0,0.00020,0.00020,0.00020,0.00020
практически,14,ADV,1.0,0.00014,0.00014,0.00014,0.00014
вместе,11,ADV,1.0,0.00011,0.00011,0.00011,0.00011
далеко,10,ADV,1.0,0.00010,0.00010,0.00010,0.00010
смочь,9,ADV,1.0,0.00009,0.00009,0.00009,0.00009


In [11]:
print_res(putin_df[(putin_df['part'] == 'ADV')].head(8), 8)

сегодня - 33, также - 24, мочь - 22, очень - 20, практически - 14, вместе - 11, далеко - 10, смочь - 9, 


In [12]:
len(putin_df)

2350

In [13]:
ryzan_df = make_dataframe('ryzan.txt')
versus(ryzan_df)
ryzan_df.head(15)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
год,59,S,0.608247,1.0,0.00059,0.00059,0.00059
регион,36,S,3.272727,1.0,0.00036,0.00036,0.00036
рязанский,30,A=m,0.000300,1.0,0.00030,0.00030,0.00030
область,30,S,4.285714,1.0,0.00030,0.00030,0.00030
проект,29,S,1.705882,1.0,0.00029,0.00029,0.00029
2018,29,NUM=ciph,7.250000,1.0,0.00029,0.00029,0.00029
это,28,S-PRO,0.168675,1.0,0.00028,0.00028,0.00028
новый,25,A=m,0.362319,1.0,0.00025,0.00025,0.00025
район,21,S,5.250000,1.0,0.00021,0.00021,0.00021
необходимо,21,PRAEDIC,1.500000,1.0,0.00021,0.00021,0.00021


In [14]:
print_res(ryzan_df.head(42), 42)

год - 59, регион - 36, рязанский - 30, область - 30, проект - 29, 2018 - 29, это - 28, новый - 25, район - 21, необходимо - 21, человек - 20, наш - 19, развитие - 18, который - 18, работа - 17, место - 16, создание - 16, строительство - 16, поддержка - 16, должный - 15, весь - 14, число - 14, производство - 14, рязань - 14, современный - 13, 2017 - 12, компания - 12, центр - 12, ;- - 12, направление - 11, задача - 11, 5 - 11, объект - 11, позволять - 11, областной - 11, сделать - 10, социальный - 10, становиться - 10, общественный - 10, качество - 10, реализация - 10, федеральный - 10, 


In [15]:
#топ прилагательных
ryzan_df[(ryzan_df['part'] == 'A') | (ryzan_df['part'] == 'A=m')
        | (ryzan_df['part'] == 'A=f') | (ryzan_df['part'] == 'A=n') | (ryzan_df['part'] == 'A=sg')
        | (ryzan_df['part'] == 'A=pl') | (ryzan_df['part'] == 'A-PRO=pl') | (ryzan_df['part'] == 'A=comp2')].head(15)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
рязанский,30,A=m,0.000300,1.0,0.00030,0.00030,0.00030
новый,25,A=m,0.362319,1.0,0.00025,0.00025,0.00025
должный,15,A=m,0.245902,1.0,0.00015,0.00015,0.00015
современный,13,A=m,0.371429,1.0,0.00013,0.00013,0.00013
областной,11,A=f,0.000110,1.0,0.00011,0.00011,0.00011
социальный,10,A=m,0.714286,1.0,0.00010,0.00010,0.00010
общественный,10,A=m,3.333333,1.0,0.00010,0.00010,0.00010
федеральный,10,A=m,3.333333,1.0,0.00010,0.00010,0.00010
хороший,8,A=m,1.333333,1.0,0.00008,0.00008,0.00008
цифровой,8,A=f,0.666667,1.0,0.00008,0.00008,0.00008


In [16]:
print_res(ryzan_df[(ryzan_df['part'] == 'A') | (ryzan_df['part'] == 'A=m')
        | (ryzan_df['part'] == 'A=f') | (ryzan_df['part'] == 'A=n') | (ryzan_df['part'] == 'A=sg')
        | (ryzan_df['part'] == 'A=pl') | (ryzan_df['part'] == 'A-PRO=pl') | (ryzan_df['part'] == 'A=comp2')].head(16), 16)

рязанский - 30, новый - 25, должный - 15, современный - 13, областной - 11, социальный - 10, общественный - 10, федеральный - 10, хороший - 8, цифровой - 8, важный - 7, общий - 7, инвестиционный - 7, основной - 6, перспективный - 6, бережливый - 6, 


In [17]:
#топ существительных
ryzan_df[(ryzan_df['part'] == 'S') | (ryzan_df['part'] == 'S-PRO')].head(16)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
год,59,S,0.608247,1.0,0.00059,0.00059,0.00059
регион,36,S,3.272727,1.0,0.00036,0.00036,0.00036
область,30,S,4.285714,1.0,0.00030,0.00030,0.00030
проект,29,S,1.705882,1.0,0.00029,0.00029,0.00029
это,28,S-PRO,0.168675,1.0,0.00028,0.00028,0.00028
район,21,S,5.250000,1.0,0.00021,0.00021,0.00021
человек,20,S,0.384615,1.0,0.00020,0.00020,0.00020
развитие,18,S,0.382979,1.0,0.00018,0.00018,0.00018
работа,17,S,0.850000,1.0,0.00017,0.00017,0.00017
место,16,S,1.777778,1.0,0.00016,0.00016,0.00016


In [18]:
print_res(ryzan_df[(ryzan_df['part'] == 'S') | (ryzan_df['part'] == 'S-PRO')].head(16), 16)

год - 59, регион - 36, область - 30, проект - 29, это - 28, район - 21, человек - 20, развитие - 18, работа - 17, место - 16, создание - 16, строительство - 16, поддержка - 16, число - 14, производство - 14, компания - 12, 


In [19]:
#топ глаголов
ryzan_df[(ryzan_df['part'] == 'V')].head(16)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
рязань,14,V,0.000140,1.0,0.00014,0.00014,0.00014
позволять,11,V,0.478261,1.0,0.00011,0.00011,0.00011
сделать,10,V,0.322581,1.0,0.00010,0.00010,0.00010
становиться,10,V,0.454545,1.0,0.00010,0.00010,0.00010
счет,9,V,1.500000,1.0,0.00009,0.00009,0.00009
рф,9,V,0.000090,1.0,0.00009,0.00009,0.00009
обеспечивать,8,V,0.258065,1.0,0.00008,0.00008,0.00008
построить,8,V,0.000080,1.0,0.00008,0.00008,0.00008
являться,8,V,1.142857,1.0,0.00008,0.00008,0.00008
увеличивать,7,V,1.750000,1.0,0.00007,0.00007,0.00007


In [20]:
print_res(ryzan_df[(ryzan_df['part'] == 'V')].head(20), 20)

рязань - 14, позволять - 11, сделать - 10, становиться - 10, счет - 9, рф - 9, обеспечивать - 8, построить - 8, являться - 8, увеличивать - 7, объем - 7, продолжать - 7, создавать - 7, получать - 7, начинать - 6, хотеть - 5, принимать - 5, ставить - 5, оставаться - 5, определять - 4, 


In [21]:
#топ наречий
ryzan_df[(ryzan_df['part'] == 'ADV')].head(8)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
наиболее,7,ADV,0.000070,1.0,0.00007,0.00007,0.00007
также,6,ADV,0.250000,1.0,0.00006,0.00006,0.00006
максимально,5,ADV,2.500000,1.0,0.00005,0.00005,0.00005
мало,5,ADV,0.833333,1.0,0.00005,0.00005,0.00005
социально,4,ADV,4.000000,1.0,0.00004,0.00004,0.00004
активно,4,ADV,0.666667,1.0,0.00004,0.00004,0.00004
вместе,3,ADV,0.272727,1.0,0.00003,0.00003,0.00003
эффективно,3,ADV,3.000000,1.0,0.00003,0.00003,0.00003


In [22]:
print_res(ryzan_df[(ryzan_df['part'] == 'ADV')].head(8), 8)

наиболее - 7, также - 6, максимально - 5, мало - 5, социально - 4, активно - 4, вместе - 3, эффективно - 3, 


In [23]:
len(ryzan_df)

1153

In [24]:
tula_df = make_dataframe('tula.txt')
versus(tula_df)
tula_df.head(15)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
год,79,S,0.814433,1.338983,1.0,0.00079,0.00079
это,40,S-PRO,0.240964,1.428571,1.0,0.00040,0.00040
область,35,S,5.000000,1.166667,1.0,0.00035,0.00035
наш,35,A-PRO=m,0.384615,1.842105,1.0,0.00035,0.00035
работа,34,S,1.700000,2.000000,1.0,0.00034,0.00034
проект,32,S,1.882353,1.103448,1.0,0.00032,0.00032
2019,30,NUM=ciph,15.000000,0.000300,1.0,0.00030,0.00030
бюджет,29,S,9.666667,29.000000,1.0,0.00029,0.00029
весь,28,A-PRO=m,0.294737,2.000000,1.0,0.00028,0.00028
новый,28,A=m,0.405797,1.120000,1.0,0.00028,0.00028


In [25]:
print_res(tula_df.head(42), 42)

год - 79, это - 40, область - 35, наш - 35, работа - 34, проект - 32, 2019 - 30, бюджет - 29, весь - 28, новый - 28, рубль - 28, который - 23, развитие - 22, регион - 22, тульский - 21, региональный - 19, становиться - 19, гражданин - 18, млн - 17, 2018 - 16, социальный - 16, число - 16, задача - 15, *** - 15, человек - 14, система - 14, тула - 14, житель - 13, результат - 13, программа - 13, центр - 13, свой - 13, должный - 13, проблема - 13, правительство - 12, рост - 12, уровень - 12, давать - 12, первый - 12, президент - 11, качество - 11, тысяча - 11, 


In [26]:
#топ прилагательных
tula_df[(tula_df['part'] == 'A') | (tula_df['part'] == 'A=m')
        | (tula_df['part'] == 'A=f') | (tula_df['part'] == 'A=n') | (tula_df['part'] == 'A=sg')
        | (tula_df['part'] == 'A=pl') | (tula_df['part'] == 'A-PRO=pl') | (tula_df['part'] == 'A=comp2')].head(15)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
новый,28,A=m,0.405797,1.120000,1.0,0.00028,0.00028
тульский,21,A=m,0.000210,0.000210,1.0,0.00021,0.00021
региональный,19,A=m,3.800000,6.333333,1.0,0.00019,0.00019
социальный,16,A=m,1.142857,1.600000,1.0,0.00016,0.00016
должный,13,A=m,0.213115,0.866667,1.0,0.00013,0.00013
федеральный,9,A=m,3.000000,0.900000,1.0,0.00009,0.00009
детский,9,A=m,2.250000,1.500000,1.0,0.00009,0.00009
медицинский,9,A=m,3.000000,3.000000,1.0,0.00009,0.00009
большой,8,A=m,0.400000,4.000000,1.0,0.00008,0.00008
муниципальный,8,A=m,8.000000,1.600000,1.0,0.00008,0.00008


In [27]:
print_res(tula_df[(tula_df['part'] == 'A') | (tula_df['part'] == 'A=m')
        | (tula_df['part'] == 'A=f') | (tula_df['part'] == 'A=n') | (tula_df['part'] == 'A=sg')
        | (tula_df['part'] == 'A=pl') | (tula_df['part'] == 'A-PRO=pl') | (tula_df['part'] == 'A=comp2')].head(16), 16)

новый - 28, тульский - 21, региональный - 19, социальный - 16, должный - 13, федеральный - 9, детский - 9, медицинский - 9, большой - 8, муниципальный - 8, областной - 7, городской - 7, единый - 6, полный - 6, национальный - 6, государственный - 6, 


In [28]:
#топ существительных
tula_df[(tula_df['part'] == 'S') | (tula_df['part'] == 'S-PRO')].head(16)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
год,79,S,0.814433,1.338983,1.0,0.00079,0.00079
это,40,S-PRO,0.240964,1.428571,1.0,0.00040,0.00040
область,35,S,5.000000,1.166667,1.0,0.00035,0.00035
работа,34,S,1.700000,2.000000,1.0,0.00034,0.00034
проект,32,S,1.882353,1.103448,1.0,0.00032,0.00032
бюджет,29,S,9.666667,29.000000,1.0,0.00029,0.00029
рубль,28,S,3.500000,4.666667,1.0,0.00028,0.00028
развитие,22,S,0.468085,1.222222,1.0,0.00022,0.00022
регион,22,S,2.000000,0.611111,1.0,0.00022,0.00022
гражданин,18,S,0.750000,6.000000,1.0,0.00018,0.00018


In [29]:
print_res(tula_df[(tula_df['part'] == 'S') | (tula_df['part'] == 'S-PRO')].head(16), 16)

год - 79, это - 40, область - 35, работа - 34, проект - 32, бюджет - 29, рубль - 28, развитие - 22, регион - 22, гражданин - 18, млн - 17, число - 16, задача - 15, человек - 14, система - 14, житель - 13, 


In [30]:
#топ глаголов
tula_df[(tula_df['part'] == 'V')].head(16)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
становиться,19,V,0.863636,1.900000,1.0,0.00019,0.00019
тула,14,V,0.000140,0.000140,1.0,0.00014,0.00014
давать,12,V,1.000000,6.000000,1.0,0.00012,0.00012
начинать,10,V,1.666667,1.666667,1.0,0.00010,0.00010
создавать,10,V,0.384615,1.428571,1.0,0.00010,0.00010
расти,9,V,1.125000,0.000090,1.0,0.00009,0.00009
использовать,8,V,0.727273,2.000000,1.0,0.00008,0.00008
принимать,7,V,1.166667,1.400000,1.0,0.00007,0.00007
объем,7,V,0.700000,1.000000,1.0,0.00007,0.00007
искать,7,V,7.000000,0.000070,1.0,0.00007,0.00007


In [31]:
print_res(tula_df[(tula_df['part'] == 'V')].head(20), 20)

становиться - 19, тула - 14, давать - 12, начинать - 10, создавать - 10, расти - 9, использовать - 8, принимать - 7, объем - 7, искать - 7, привлекать - 7, предусматривать - 6, хотеть - 6, выполнять - 6, развивать - 6, работать - 6, отмечать - 6, составлять - 6, получать - 5, позволять - 5, 


In [32]:
#топ наречий
tula_df[(tula_df['part'] == 'ADV')].head(8)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
сегодня,9,ADV,0.272727,9.00000,1.0,0.00009,0.00009
мочь,6,ADV,0.272727,6.00000,1.0,0.00006,0.00006
активно,6,ADV,1.000000,1.50000,1.0,0.00006,0.00006
менее,4,ADV,1.000000,4.00000,1.0,0.00004,0.00004
новомосковск,4,ADV,0.000040,0.00004,1.0,0.00004,0.00004
достойно,3,ADV,3.000000,0.00003,1.0,0.00003,0.00003
вдвое,3,ADV,1.500000,0.00003,1.0,0.00003,0.00003
очень,3,ADV,0.150000,1.00000,1.0,0.00003,0.00003


In [33]:
print_res(tula_df[(tula_df['part'] == 'ADV')].head(8), 8)

сегодня - 9, мочь - 6, активно - 6, менее - 4, новомосковск - 4, достойно - 3, вдвое - 3, очень - 3, 


In [34]:
len(tula_df)

1343

In [35]:
tumen_df = make_dataframe('tumen.txt')
versus(tumen_df)
tumen_df.head(15)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
весь,32,A-PRO=m,0.336842,2.285714,1.142857,1.0,0.00032
наш,32,A-PRO=m,0.351648,1.684211,0.914286,1.0,0.00032
тюменский,27,A=m,0.000270,0.000270,0.000270,1.0,0.00027
новый,24,A=m,0.347826,0.960000,0.857143,1.0,0.00024
область,24,S,3.428571,0.800000,0.685714,1.0,0.00024
это,24,S-PRO,0.144578,0.857143,0.600000,1.0,0.00024
который,23,A-PRO=m,0.315068,1.277778,1.000000,1.0,0.00023
должный,22,A=m,0.360656,1.466667,1.692308,1.0,0.00022
год,19,S,0.195876,0.322034,0.240506,1.0,0.00019
проект,18,S,1.058824,0.620690,0.562500,1.0,0.00018


In [36]:
print_res(tumen_df.head(42), 42)

весь - 32, наш - 32, тюменский - 27, новый - 24, область - 24, это - 24, который - 23, должный - 22, год - 19, проект - 18, становиться - 17, развитие - 17, друг - 14, экономика - 13, каждый - 12, человек - 12, социальный - 12, гражданин - 10, россия - 10, свой - 10, качество - 10, жизнь - 10, задача - 10, общество - 9, уровень - 9, — - 9, равенство - 9, сделать - 8, возможность - 8, поддержка - 8, продолжать - 8, национальный - 7, говорить - 7, рост - 7, создавать - 7, общественный - 7, регион - 7, труд - 7, инициатива - 7, медицинский - 7, внимание - 7, главный - 7, 


In [37]:
#топ прилагательных
tumen_df[(tumen_df['part'] == 'A') | (tumen_df['part'] == 'A=m')
        | (tumen_df['part'] == 'A=f') | (tumen_df['part'] == 'A=n') | (tumen_df['part'] == 'A=sg')
        | (tumen_df['part'] == 'A=pl') | (tumen_df['part'] == 'A-PRO=pl') | (tumen_df['part'] == 'A=comp2')].head(15)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
тюменский,27,A=m,0.000270,0.000270,0.000270,1.0,0.00027
новый,24,A=m,0.347826,0.960000,0.857143,1.0,0.00024
должный,22,A=m,0.360656,1.466667,1.692308,1.0,0.00022
социальный,12,A=m,0.857143,1.200000,0.750000,1.0,0.00012
национальный,7,A=m,1.000000,2.333333,1.166667,1.0,0.00007
общественный,7,A=m,2.333333,0.700000,1.400000,1.0,0.00007
медицинский,7,A=m,2.333333,2.333333,0.777778,1.0,0.00007
главный,7,A=m,1.400000,3.500000,1.400000,1.0,0.00007
региональный,6,A=m,1.200000,2.000000,0.315789,1.0,0.00006
хороший,6,A=m,1.000000,0.750000,1.500000,1.0,0.00006


In [38]:
print_res(tumen_df[(tumen_df['part'] == 'A') | (tumen_df['part'] == 'A=m')
        | (tumen_df['part'] == 'A=f') | (tumen_df['part'] == 'A=n') | (tumen_df['part'] == 'A=sg')
        | (tumen_df['part'] == 'A=pl') | (tumen_df['part'] == 'A-PRO=pl') | (tumen_df['part'] == 'A=comp2')].head(16), 16)

тюменский - 27, новый - 24, должный - 22, социальный - 12, национальный - 7, общественный - 7, медицинский - 7, главный - 7, региональный - 6, хороший - 6, крупный - 6, близкий - 6, справедливый - 6, равный - 6, малый - 6, технологический - 5, 


In [39]:
#топ существительных
tumen_df[(tumen_df['part'] == 'S') | (tumen_df['part'] == 'S-PRO')].head(16)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
область,24,S,3.428571,0.800000,0.685714,1.0,0.00024
это,24,S-PRO,0.144578,0.857143,0.600000,1.0,0.00024
год,19,S,0.195876,0.322034,0.240506,1.0,0.00019
проект,18,S,1.058824,0.620690,0.562500,1.0,0.00018
развитие,17,S,0.361702,0.944444,0.772727,1.0,0.00017
друг,14,S-PRO,3.500000,4.666667,7.000000,1.0,0.00014
экономика,13,S,0.619048,13.000000,3.250000,1.0,0.00013
человек,12,S,0.230769,0.600000,0.857143,1.0,0.00012
гражданин,10,S,0.416667,3.333333,0.555556,1.0,0.00010
россия,10,S,0.117647,2.500000,1.111111,1.0,0.00010


In [40]:
print_res(tumen_df[(tumen_df['part'] == 'S') | (tumen_df['part'] == 'S-PRO')].head(16), 16)

область - 24, это - 24, год - 19, проект - 18, развитие - 17, друг - 14, экономика - 13, человек - 12, гражданин - 10, россия - 10, качество - 10, жизнь - 10, задача - 10, общество - 9, уровень - 9, равенство - 9, 


In [41]:
#топ глаголов
tumen_df[(tumen_df['part'] == 'V')].head(16)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
становиться,17,V,0.772727,1.700000,0.894737,1.0,0.00017
сделать,8,V,0.258065,0.800000,2.000000,1.0,0.00008
продолжать,8,V,1.142857,1.142857,1.600000,1.0,0.00008
говорить,7,V,0.350000,0.000070,3.500000,1.0,0.00007
создавать,7,V,0.269231,1.000000,0.700000,1.0,0.00007
работать,6,V,0.315789,2.000000,1.000000,1.0,0.00006
обеспечивать,6,V,0.193548,0.750000,3.000000,1.0,0.00006
давать,5,V,0.416667,2.500000,0.416667,1.0,0.00005
думать,5,V,1.000000,0.000050,5.000000,1.0,0.00005
оставаться,5,V,0.555556,1.000000,5.000000,1.0,0.00005


In [42]:
print_res(tumen_df[(tumen_df['part'] == 'V')].head(20), 20)

становиться - 17, сделать - 8, продолжать - 8, говорить - 7, создавать - 7, работать - 6, обеспечивать - 6, давать - 5, думать - 5, оставаться - 5, видеть - 4, являться - 4, развивать - 4, показывать - 3, ставить - 3, находиться - 3, объем - 3, капитал - 3, запускать - 3, построить - 3, 


In [43]:
#топ наречий
tumen_df[(tumen_df['part'] == 'ADV')].head(8)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
округ,6,ADV,6.000000,0.00006,0.000060,1.0,0.00006
прежде,5,ADV,0.714286,2.50000,5.000000,1.0,0.00005
мочь,5,ADV,0.227273,5.00000,0.833333,1.0,0.00005
научно,4,ADV,4.000000,4.00000,0.000040,1.0,0.00004
мало,3,ADV,0.500000,0.60000,1.500000,1.0,0.00003
активно,3,ADV,0.500000,0.75000,0.500000,1.0,0.00003
действительно,3,ADV,1.000000,0.00003,0.000030,1.0,0.00003
тобольск,3,ADV,0.000030,0.00003,0.000030,1.0,0.00003


In [44]:
print_res(tumen_df[(tumen_df['part'] == 'ADV')].head(8), 8)

округ - 6, прежде - 5, мочь - 5, научно - 4, мало - 3, активно - 3, действительно - 3, тобольск - 3, 


In [45]:
len(tumen_df)

1177

In [46]:
yamal_df = make_dataframe('yamal.txt')
versus(yamal_df)
yamal_df.head(15)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
год,96,S,0.989691,1.627119,1.215190,5.052632,1.0
это,48,S-PRO,0.289157,1.714286,1.200000,2.000000,1.0
округ,37,ADV,37.000000,0.000370,0.000370,6.166667,1.0
ямал,36,V,0.000360,0.000360,0.000360,36.000000,1.0
регион,31,S,2.818182,0.861111,1.409091,4.428571,1.0
наш,30,A-PRO=m,0.329670,1.578947,0.857143,0.937500,1.0
весь,30,A-PRO=m,0.315789,2.142857,1.071429,0.937500,1.0
развитие,26,S,0.553191,1.444444,1.181818,1.529412,1.0
новый,26,A=m,0.376812,1.040000,0.928571,1.083333,1.0
россия,26,S,0.305882,6.500000,2.888889,2.600000,1.0


In [47]:
print_res(yamal_df.head(42), 42)

год - 96, это - 48, округ - 37, ямал - 36, регион - 31, наш - 30, весь - 30, развитие - 26, новый - 26, россия - 26, работа - 26, проект - 23, становиться - 21, ямалец - 18, программа - 17, строительство - 17, число - 16, социальный - 16, принимать - 15, поддержка - 15, страна - 14, арктический - 14, жилье - 14, территория - 14, житель - 14, который - 14, оставаться - 14, объект - 14, сегодня - 13, рубль - 13, предприятие - 13, следующий - 13, возможность - 13, сфера - 13, свой - 13, задача - 13, ребенок - 13, депутат - 12, внимание - 12, человек - 12, правительство - 12, проводить - 12, 


In [48]:
#топ прилагательных
yamal_df[(yamal_df['part'] == 'A') | (yamal_df['part'] == 'A=m')
        | (yamal_df['part'] == 'A=f') | (yamal_df['part'] == 'A=n') | (yamal_df['part'] == 'A=sg')
        | (yamal_df['part'] == 'A=pl') | (yamal_df['part'] == 'A-PRO=pl') | (yamal_df['part'] == 'A=comp2')].head(15)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
новый,26,A=m,0.376812,1.040000,0.928571,1.083333,1.0
социальный,16,A=m,1.142857,1.600000,1.000000,1.333333,1.0
арктический,14,A=m,14.000000,0.000140,0.000140,0.000140,1.0
следующий,13,A=m,1.625000,13.000000,2.600000,6.500000,1.0
северный,11,A=m,5.500000,5.500000,11.000000,5.500000,1.0
большой,11,A=m,0.550000,5.500000,1.375000,5.500000,1.0
уважаемый,10,A=m,0.526316,2.500000,0.000100,5.000000,1.0
федеральный,10,A=m,3.333333,1.000000,1.111111,2.500000,1.0
текущий,9,A=m,9.000000,0.000090,9.000000,9.000000,1.0
международный,9,A=m,0.900000,4.500000,4.500000,0.000090,1.0


In [49]:
print_res(yamal_df[(yamal_df['part'] == 'A') | (yamal_df['part'] == 'A=m')
        | (yamal_df['part'] == 'A=f') | (yamal_df['part'] == 'A=n') | (yamal_df['part'] == 'A=sg')
        | (yamal_df['part'] == 'A=pl') | (yamal_df['part'] == 'A-PRO=pl') | (yamal_df['part'] == 'A=comp2')].head(16), 16)

новый - 26, социальный - 16, арктический - 14, следующий - 13, северный - 11, большой - 11, уважаемый - 10, федеральный - 10, текущий - 9, международный - 9, современный - 9, коренной - 9, морской - 8, крупный - 8, дорогой - 7, последний - 7, 


In [50]:
#топ существительных
yamal_df[(yamal_df['part'] == 'S') | (yamal_df['part'] == 'S-PRO')].head(16)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
год,96,S,0.989691,1.627119,1.215190,5.052632,1.0
это,48,S-PRO,0.289157,1.714286,1.200000,2.000000,1.0
регион,31,S,2.818182,0.861111,1.409091,4.428571,1.0
развитие,26,S,0.553191,1.444444,1.181818,1.529412,1.0
россия,26,S,0.305882,6.500000,2.888889,2.600000,1.0
работа,26,S,1.300000,1.529412,0.764706,8.666667,1.0
проект,23,S,1.352941,0.793103,0.718750,1.277778,1.0
ямалец,18,S,0.000180,0.000180,0.000180,0.000180,1.0
программа,17,S,1.888889,2.833333,1.307692,4.250000,1.0
строительство,17,S,1.416667,1.062500,2.125000,5.666667,1.0


In [51]:
print_res(yamal_df[(yamal_df['part'] == 'S') | (yamal_df['part'] == 'S-PRO')].head(16), 16)

год - 96, это - 48, регион - 31, развитие - 26, россия - 26, работа - 26, проект - 23, ямалец - 18, программа - 17, строительство - 17, число - 16, поддержка - 15, страна - 14, жилье - 14, территория - 14, житель - 14, 


In [52]:
#топ глаголов
yamal_df[(yamal_df['part'] == 'V')].head(16)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
ямал,36,V,0.000360,0.000360,0.000360,36.000000,1.0
становиться,21,V,0.954545,2.100000,1.105263,1.235294,1.0
принимать,15,V,2.500000,3.000000,2.142857,7.500000,1.0
оставаться,14,V,1.555556,2.800000,14.000000,2.800000,1.0
проводить,12,V,1.200000,4.000000,4.000000,6.000000,1.0
продолжать,11,V,1.571429,1.571429,2.200000,1.375000,1.0
благодарить,10,V,10.000000,0.000100,10.000000,0.000100,1.0
поручать,9,V,0.000090,9.000000,4.500000,9.000000,1.0
вырастать,8,V,2.000000,0.000080,8.000000,4.000000,1.0
счет,8,V,1.333333,0.888889,0.000080,0.000080,1.0


In [53]:
print_res(yamal_df[(yamal_df['part'] == 'V')].head(20), 20)

ямал - 36, становиться - 21, принимать - 15, оставаться - 14, проводить - 12, продолжать - 11, благодарить - 10, поручать - 9, вырастать - 8, счет - 8, планировать - 8, составлять - 7, создавать - 7, давать - 7, работать - 7, начинать - 7, вестись - 7, удаваться - 7, построить - 6, являться - 6, 


In [54]:
#топ наречий
yamal_df[(yamal_df['part'] == 'ADV')].head(8)

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
округ,37,ADV,37.000000,0.00037,0.000370,6.166667,1.0
сегодня,13,ADV,0.393939,13.00000,1.444444,6.500000,1.0
свыше,8,ADV,4.000000,8.00000,8.000000,0.000080,1.0
значительно,5,ADV,1.250000,0.00005,1.666667,0.000050,1.0
совместно,5,ADV,2.500000,2.50000,2.500000,0.000050,1.0
несмотря,4,ADV,1.333333,0.00004,0.000040,0.000040,1.0
темп,4,ADV,0.571429,2.00000,4.000000,2.000000,1.0
молодежь,4,ADV,1.000000,2.00000,4.000000,0.000040,1.0


In [55]:
print_res(yamal_df[(yamal_df['part'] == 'ADV')].head(8), 8)

округ - 37, сегодня - 13, свыше - 8, значительно - 5, совместно - 5, несмотря - 4, темп - 4, молодежь - 4, 


In [56]:
len(yamal_df)

1600

In [57]:
ryzan_df.sort_values(by=['vs_yamal'])

,count,part,vs_putin,vs_ryzan,vs_tula,vs_tumen,vs_yamal
неделя,1,S,1.000000,1.0,0.00001,0.00001,0.00001
220,1,NUM=ciph,0.000010,1.0,0.00001,0.00001,0.00001
детсад,1,S,0.000010,1.0,0.00001,0.00001,0.00001
250,1,NUM=ciph,0.500000,1.0,0.00001,0.00001,0.00001
132,1,NUM=ciph,0.000010,1.0,0.00001,0.00001,0.00001
скопинский,1,A=m,0.000010,1.0,0.00001,0.00001,0.00001
ильинка,1,S,0.000010,1.0,0.00001,0.00001,0.00001
канищево,1,ADV,0.000010,1.0,0.00001,0.00001,0.00001
дпр,1,V,0.000010,1.0,0.00001,0.00001,0.00001
730,1,NUM=ciph,0.000010,1.0,0.00001,0.00001,0.00001


In [58]:
#import gensim
import string
import pyphen
import pprint
from nltk.tokenize import sent_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [59]:
def FRE(text):
    words = []
    syls = []
    dic = pyphen.Pyphen(lang='ru')
    f = open(text)
    txt = ''
    for line in f:
        txt = txt + line.replace('\n', '')

    words = txt.split(' ')
    sents = sent_tokenize(txt, 'russian')
    for i in words:
        syls = syls + dic.inserted(i).split('-')
        
    fre = 206.835 -1.3 * len(words)/len(sents) - 60.1 * len(syls)/len(words)
    return fre

In [60]:
FRE('putin.txt')

29.084170005532854

In [61]:
FRE('ryzan.txt')

27.816930081761257

In [62]:
FRE('tula.txt')

30.920437245129136

In [63]:
FRE('tumen.txt')

30.243864285785747

In [64]:
FRE('yamal.txt')

31.862611311141052

In [65]:
data = yamal_df
print('прилагательных: ', len(data[(data['part'] == 'A') | (data['part'] == 'A=m')
        | (data['part'] == 'A=f') | (data['part'] == 'A=n') | (data['part'] == 'A=sg')
        | (data['part'] == 'A=pl') | (data['part'] == 'A-PRO=pl') | (data['part'] == 'A=comp2')]),
      'существительных: ', len(data[(data['part'] == 'S') | (data['part'] == 'S-PRO')]),
      'глаголов: ', len(data[(data['part'] == 'V')]),
      'наречий: ', len(data[(data['part'] == 'ADV')]))

прилагательных:  345 существительных:  695 глаголов:  347 наречий:  88


In [66]:
def grams(file_name, k):
    #names_list = ['putin.txt', 'ryzan.txt', 'tula.txt', 'tumen.txt', 'yamal.txt']
    names_list = ['ryzan.txt', 'tula.txt', 'tumen.txt', 'yamal.txt']
    words = []
    dicts = []
    f = open(file_name)
    txt = ''
    for line in f:
        txt = txt + line.replace('\n', '')
    
    array = tokenization(txt).split(' ')
    #array = txt.split(' ')
    words = words + array
    gr = list(ngrams(words, k))
    
    names_list.remove(file_name)
    
    for i in names_list:
        words = []
        res = []
        f = open(i)
        txt = ''
        for line in f:
            txt = txt + line.replace('\n', '')
    
        array = tokenization(txt).split(' ')
        #array = txt.split(' ')
        words = words + array
        
        gr1 = list(ngrams(words, k))
        gr1_0 = gr1
        
        for j in gr:
            while j in gr1:
                res = res + [j]
                gr1.remove(j)
        
        print('\n', file_name, ' vs ', i, ': ')
        d = dict(Counter(res).most_common())
        pprint.pprint(d)
        
        dicts = dicts + [d]
        
    common_keys = set(dicts[0].keys())
    for d in dicts[1:]:
        common_keys.intersection_update(set(d.keys()))
        
    return common_keys

In [67]:
#for q in ['putin.txt', 'ryzan.txt', 'tula.txt', 'tumen.txt', 'yamal.txt']:
for q in ['ryzan.txt', 'tula.txt', 'tumen.txt', 'yamal.txt']:
    print('common: ', grams(q, 2))
    print('###############################################################################################################')


 ryzan.txt  vs  tula.txt : 
{('', ''): 2,
 ('', '(«'): 1,
 ('', '«'): 28,
 ('', 'внедрение'): 1,
 ('', 'конец'): 1,
 ('', 'который'): 1,
 ('', 'повышение'): 1,
 ('', 'поддержка'): 1,
 ('', 'промышленный'): 1,
 ('', 'развитие'): 2,
 ('', 'реализация'): 1,
 ('', 'создание'): 1,
 ('', 'улучшать'): 2,
 ('', 'человек'): 1,
 ('', 'это'): 9,
 ('', '–'): 91,
 ('100', 'млрд'): 1,
 ('2017', 'год'): 2,
 ('2018', 'год'): 16,
 ('5', 'млн'): 1,
 ('«', 'бережливый'): 1,
 ('«', 'зеленый'): 1,
 ('«', 'открытый'): 1,
 ('«', 'патриот'): 1,
 ('«', 'формирование'): 1,
 ('»', ''): 13,
 ('»', 'необходимо'): 1,
 ('»,', ''): 4,
 ('активно', 'включаться'): 1,
 ('актуальный', 'проблема'): 2,
 ('безопасный', 'качественный'): 1,
 ('бережливый', 'поликлиника'): 1,
 ('близкий', 'год'): 1,
 ('весь', 'план'): 2,
 ('весь', 'свой'): 1,
 ('весь', 'страна'): 1,
 ('включаться', 'весь'): 1,
 ('власть', 'муниципальный'): 1,
 ('власть', 'общество'): 1,
 ('волонтерский', 'движение'): 1,
 ('востребовать', 'специальность'): 1,


In [68]:
words = []

f = open('yamal.txt')
txt = ''
for line in f:
    txt = txt + line.replace('\n', '')
    
array = tokenization(txt).split(' ')
words = words + array

dict(Counter(list(ngrams(words, 4))).most_common())

{('', '–', '', 'это'): 5,
 ('ямал', '', '–', ''): 3,
 ('надым', '', '–', ''): 3,
 ('', '–', '', 'салехард'): 3,
 ('уважаемый', 'сергей', 'миронович', 'депутат'): 2,
 ('страна', '', '–', ''): 2,
 ('перевозка', 'северный', 'морской', 'путь'): 2,
 ('', '–', '', 'исключение'): 2,
 ('завод', '', '«', 'ямал'): 2,
 ('', '«', 'ямал', 'спг'): 2,
 ('«', 'ямал', 'спг', '»'): 2,
 ('близкий', 'план', '', '–'): 2,
 ('план', '', '–', ''): 2,
 ('', '–', '', 'строительство'): 2,
 ('', '–', '', 'создание'): 2,
 ('программа', '', '«', 'сотрудничество'): 2,
 ('', '«', 'сотрудничество', '»'): 2,
 ('поручать', 'первый', 'заместитель', 'губернатор'): 2,
 ('первый', 'заместитель', 'губернатор', 'алексей'): 2,
 ('заместитель', 'губернатор', 'алексей', 'викторович'): 2,
 ('губернатор', 'алексей', 'викторович', 'ситников'): 2,
 ('', '«', 'ямал', '»'): 2,
 ('«', 'ямал', '»', ''): 2,
 ('цель', '', '–', ''): 2,
 ('', '–', '', 'повышение'): 2,
 ('программа', 'переселение', 'ветхий', 'аварийный'): 2,
 ('переселение',

In [69]:
words = []
f = open('yamal.txt')
txt = ''
for line in f:
    txt = txt + line.replace('\n', '')
    
array = txt.split(' ')
words = words + array

In [70]:
len(words)

5326